In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
ratings = pd.read_csv("data/ratings.csv")
movies = pd.read_csv("data\movies.csv")

In [ ]:
movie_ratings = pd.merge(ratings,movies,on='movieId')
movie_ratings

In [ ]:
title_user = movie_ratings.pivot_table(index='userId', columns='title', values='rating')
title_user = title_user.fillna(0)

rating_matrix = title_user.T # 행과 열 바꾸기(전치행렬)
rating_matrix

In [ ]:
item_sim = cosine_similarity(rating_matrix,rating_matrix)
item_sim_df = pd.DataFrame(data=item_sim,index=rating_matrix.index, columns=rating_matrix.index) # numpy array에 바로 집어 넣은 것
item_sim_df

In [ ]:
item_sim_df['African Queen, The (1951)'].sort_values(ascending=False)[1:6]

# 서프라이즈 라이브러리 사용

설치 안되면, Microsoft C++ Build Tools - Visual Studio -> 여기서 Tool 설치

<numpy 에러 발생시>
uv pip uninstall numpy
uv pip install "numpy<2"

아니면 파이썬 버전 3.10 으로 다운 그레이드 하고 진행

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.dataset import DatasetAutoFolds
import pandas as pd

In [ ]:
ratings = pd.read_csv("data/ratings.csv")
ratings.head()

In [ ]:
ratings.to_csv("data/ratings_nohead.csv", index= False, header=False)

In [ ]:
ratings.describe()

In [ ]:
ratings['rating'].unique()

In [ ]:
reader = Reader(line_format = 'user item rating timestamp',sep=',',rating_scale=(0.5,5)) # line_format: csv 파일의 컬럼명 지정, sep 을 넣어주는 이유는 ,로 구분이 안되어있는 것들이 있어서.
#  rating_scale=(0.5,5) -> 크기가 0.5가 최소 5가 최대
data_folds = DatasetAutoFolds(ratings_file='data/ratings_nohead.csv',reader=reader) # ratings_file : 내가 학습시킬 파일
trainset = data_folds.build_full_trainset() #학습용 100%로 하겠다

# 모델 생성
model = SVD(n_factors=50, random_state=42) #적절한 n_factors 를 제공해주는 것이 좋음.
model.fit(trainset)



In [ ]:
y_pred = model.predict('9','3') # 9번:userid 유저의 3번 영화
y_pred

## 유저가 영화를 봤는 지 안봤는 지 확인
### 전체 영화 - 본 영화 = 안 본 영화

In [ ]:
movies = pd.read_csv('data/movies.csv')

def get_unseen_surprise(userid):
    return list(set(movies['movieId']) - set(ratings[ratings['userId'] == userid]['movieId']))

unseen_movies = get_unseen_surprise(9)

In [ ]:
pred_result = []

for movieid in unseen_movies:
    result = model.predict('9',str(movieid))
    data = {}  #{'id':1, 'rating':3.5, 'title':'Avatar'}
    data['id'] = result.iid # 내가 평점을 매긴 id
    data['rating'] = result.est 
    data['title'] = movies[movies['movieId']==int(result.iid)]['title'].iloc[0]
    pred_result.append(data)

result_df = pd.DataFrame(pred_result) 
result_df.sort_values('rating',ascending=False)[0:5]

In [ ]:
result_df[result_df['title']=='African Queen, The (1951)']

# autoviz

In [ ]:
import seaborn as sns

In [ ]:
df = pd.read_csv("data/heart.csv")
df 

In [ ]:
%matplotlib inline
from autoviz.AutoViz_Class import AutoViz_Class #Auto ML 쓰는 이유 : 젤 좋은 모델 알기 위해서.
import matplotlib.pyplot as plt

av = AutoViz_Class()

av.AutoViz(
    filename = '',
    dfte=df,
    depVar='target',
    verbose=1,
    max_rows_analyzed=df.shape[0],
    max_cols_analyzed=df.shape[1]
)

In [ ]:
from sklearn.model_selection import train_test_split

y_data = df.pop('target') # 함부로 쓰면 안됨. 없어지기 때문에
X_data = df
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=0.2)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
from lazypredict.Supervised import LazyClassifier

lcf_model = LazyClassifier(verbose=0, ignore_warnings=True,custom_metric=None)
models,predictions = lcf_model.fit(X_train,X_test,y_train,y_test)

In [ ]:
models

In [ ]:
df = pd.read_csv('data/heart.csv')
df

# Optuna 사용 -> 최적화된 하이퍼 파라미터를 설정해줌

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
import optuna
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()

X_data = data.data
y_data = data.target

X_train,X_test,y_train, y_test = train_test_split(X_data,y_data,test_size=0.2,random_state=42) # 모델이 랜덤 값에 영향을 받기 때문에 고정해놔야 함.

In [ ]:
def objective(trial):
    model_type = trial.suggest_categorical("model",['RandomForest','GradientBoost','XGBoost']) # 순서대로 지정해준다는 뜻

    if model_type=="RandomForest":
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int('n_estimators',100,1500), # int형의 값을 suggest 하겠다. 100부터 1500 까지
            max_depth = trial.suggest_int('max_depth',3,50),
            min_samples_split = trial.suggest_int('min_samples_split',2,10),
            random_state = 42
        )
    elif model_type=="GradientBoost":
        model = GradientBoostingRegressor(
        n_estimators=trial.suggest_int('n_estimators',100,1500), # int형의 값을 suggest 하겠다. 100부터 1500 까지
        max_depth = trial.suggest_int('max_depth',3,50),
        learning_rate = trial.suggest_float('learning_rate',0.01,0.3),  #learning rate 값 : 0.01 => 딥러닝에선 영향을 많이 끼친다.
        random_state = 42
        )
    elif model_type=="XGBoost":
        model = XGBRegressor(
        n_estimators = trial.suggest_int('n_estimators',100,1500), # int형의 값을 suggest 하겠다. 100부터 1500 까지
        max_depth = trial.suggest_int('max_depth',3,50),
        learning_rate = trial.suggest_float('learning_rate',0.01,0.3),  #learning rate 값 : 0.01 => 딥러닝에선 영향을 많이 끼친다.
        subsample = trial.suggest_float('subsample',0.5,1.0),
        random_state = 42
        )

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    return r2_score(y_test,y_pred)
    


study = optuna.create_study(direction="maximize") # 결과값이 커지는 방향으로 파인 튜닝해라
study.optimize( objective, n_trials=50) # n_trials은 50번 학습해라.

print("최고의 모델 : ",study.best_params['model'])
print('최고의 파라미터 : ',study.best_params)
print('최고 스코어 : ',study.best_value)


In [ ]:
# # 예시
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.datasets import load_iris
# from sklearn.model_selection import train_test_split
# import joblib

# # 데이터 준비 및 모델 학습
# X, y = load_iris(return_X_y=True)
# X_train, X_test, y_train, y_test = train_test_split(X, y)
# model = RandomForestClassifier()
# model.fit(X_train, y_train)

# # 모델 저장
# joblib.dump(model, 'model.pkl')


In [ ]:
# model1 = joblib.load('model.plk') # -> 파일로 저장

In [ ]:
# model1.predict(X_test)

In [ ]:
# 이렇게 저장하면 됨
# model = GradientBoostingRegressor(
#             n_estimators=100,
#             max_depth=3,
#             learning_rate=0.01,
#             random_state=42)

# model.fit(X_train)

import joblib

joblib.dump(model,'model/best_GB_Regressor.pkl')

In [ ]:
# 모델 불러오기
loaded_model = joblib.load('model/best_GB_Regressor.pkl')
loaded_model

## pickle 사용

In [ ]:
# pickle 사용
# 모델 저장
import pickle

with open('best_GB_Regressor.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
# 모델 불러오기

with open('best_GB_Regressor.pkl', 'rb') as f:
    loaded_model = pickle.load(f)